# 목표
- 데이터에서 상위 33,000개의 샘플만 사용
- 33,000개 중에서 3000개는 테스트 데이터로 분리하여 모델을 학습, 번역을 테스트하는 용도로 사용
<br>  

### 1. 정제, 정규화, 정처리(영어, 프랑스어 둘다)
---
1. 구두점을 단어와 분리
    - ex) he is a good boy! -> 'he' 'is' 'a' 'good' 'boy' '!'
    - 그래서 결국 전처리는 'he is a good boy !' 이런식으로 하면 띄어쓰기 단위로 토큰화 수행
2. 모두 소문자로 변환
    - ```str.lower()```
3. 띄어쓰기 단위로 토큰화
    - 토크나이저..로 나누라는 소리인가 split으로 나누라는 것인가
    - ex) 'he is a good boy !' -> ['he', 'is', 'a', 'good', 'boy', '!']
  
  <br>  

### 2. 디코더 문장에 시작토큰, 종료토큰 추가
---
입력 데이터가 Courez! 였으면  
step 1을 지나면서 ['courez' '!']로 토큰화  
- 입력 시퀀스 : ['', 'courez', '!']
- 레이블 시퀀스 : ['courez', '!', '']
  
  <br>  

### 3. 케라스 토크나이저 텍스트 -> 숫자로 바꾸기
---
각 단어를 고유한 정수로 바꿔서 처리한다.  
```python
setences = [ '뭐가 막 담겨있다하고' ]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(setnences)
print(tokenizer.word_index)
# 이러면 뭔가 쫙 나옴 (dict형으로 return)
```
- ```fit_on_texts``` : 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여, 
- ```word_index``` : 어떻게 인덱스가 부여되었는지를 보려면 사용
- ```word_counts``` : 각 단어가 카운트를 수행하였을 때 몇 개 였는지를 보고자 한다면
- ```text_to_sequences()``` : 입력으로 들어온 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환한다. (**이거다**)
```python
print(tokenizer.texts_to_sequences(sentences))
# 이러면 기존에 단어로 이루어진 문장들이 숫자로 변환되어서 나온다
```
  
  <br>  

### 4. Embedding Layer 사용
---
각 단어를 Embedding Layer를 사용해서 **벡터화**시켜라.  
Embedding Layer에 필요한 인자는 3가지, vocab_size, output_dim, input_length..  
  
아래는 위키독스의 내용
```python
v = Embedding(20000, 128, input_length = 500)
# vocab_size = 20000
# ouput_dim = 128
# input_length = 500

# 토큰화 수행
t = Tokenizer()
t.fit_on_texts(sentences)

# vocab_size 구하기
vocab_size = len(t.word_index) + 1

# 가장 긴 문장 길이 구하기
# 정수 인코딩 후에 구한다
X_encoded = t.text_to_sequences(sentences)
max_len = max(len(l) for l in X_encoded)
```

이를 통해서 인코더와 디코더의 임베딩 층을 각각 구현해본다.

```python
from tensorflow.keras.layers import Input, Embedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
```
이런게 그럼 디코더에도 만들어야 한다는 것이겠지  
  
> 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사욯해야한다.
> 하지만 디코더의 Train과정과 Test과정(prediction과정)에서의 임베딩층은 동일해야 한다.

  
  <br>  

### 5. 모델 구현
### 6. 모델 평가

# 시작

## 라이브러리 import
---

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import re

## 데이터 가져오기
---
- 3번째 Column은 필요가 없기 때문에 삭제한다.
- 사용할 데이터 양은 33,000개의 샘플
    - Train : 30,000 / Test : 3,000

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
71284,It's always been that way.,Il en a toujours été ainsi.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
57872,There is one difference.,Il y a une différence.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
67959,Have you ever loved a man?,As-tu déjà aimé un homme ?,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
123767,There is a television in the room.,Il y a un poste de télévision dans la pièce.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
122375,I wonder if the weather will hold.,Je me demande si le temps se maintiendra.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 3.3만개의 샘플
lines.sample(5)

,eng,fra
13086,We're different.,Nous sommes différents.
1087,Are you up?,Es-tu levé ?
25169,I've caught a cold.,J'ai attrapé froid.
15547,I'm on the phone.,Je suis au téléphone.
24645,I saw the man jump.,J'ai vu l'homme sauter.


## 정제, 정규화, 전처리(영어, 프랑스어 둘다)
---
1. 구두점을 단어와 분리
    - ex) he is a good boy! -> 'he' 'is' 'a' 'good' 'boy' '!'
    - 그래서 결국 전처리는 'he is a good boy !' 이런식으로 하면 띄어쓰기 단위로 토큰화 수행
2. 모두 소문자로 변환
    - ```str.lower()```
3. 띄어쓰기 단위로 토큰화
    - 토크나이저..로 나누라는 소리인가 split으로 나누라는 것인가
    - ex) 'he is a good boy !' -> ['he', 'is', 'a', 'good', 'boy', '!']

In [4]:
lines.sample(6)

,eng,fra
23658,He is about to die.,Il est sur le point de mourir.
23170,Do you have a blog?,Avez-vous un blog ?
27794,You know the drill.,Vous connaissez le refrain.
17002,Was anybody hurt?,Qui que ce soit a-t-il été blessé ?
28887,He averted his gaze.,Il évita son regard.
30124,I study very little.,J'étudie très peu.


### 전처리
- 구두점 나누기
- 소문자로 만들기

In [5]:
def preprocess_sentence(sentence) :
    
    # 전처리 부분

    # 구두점을 단어와 분리를 시켜본다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-zA-Z!.?]+", r" ", sentence)
    sentence = re.sub(r"\s+", r" ", sentence)
#     sentence = re.sub(r'[" "]+', " ", sentence)
    
    # 모두 소문자로 변환
    sentence = sentence.lower()
    
    return sentence

lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence(x))
lines.sample(5)

,eng,fra
28208,alert the neighbors !,alertons les voisins !
18712,he looks after us .,il s occupe de nous .
22082,what a great view !,quelle superbe vue !
9264,they look cool .,elles ont l air d tendues .
156,i drive .,je conduis .


### 시작토큰과 종료토큰 추가하기
- 프랑스어에만 추가 (decoder 언어)
    - decoder_input에는 시작태그만
    - decoder_target에는 종료태그만 남아야 한다.

In [6]:
### 시작 토큰과 종료 토큰 추가
sos_token = '<sos>'
eos_token = '<eos>'
lines.fra = lines.fra.apply(lambda x : sos_token + ' ' + x + ' ' + eos_token)
print('전체 샘플의 수 : ', len(lines))
lines.sample(5)

전체 샘플의 수 :  33000


,eng,fra
30080,i said i d be there .,<sos> j ai dit que je serais l . <eos>
8630,is he japanese ?,<sos> est il japonais ? <eos>
6227,it s terrible .,<sos> c est terrible . <eos>
9385,tom is in town .,<sos> tom est en ville . <eos>
12466,that s not mine .,<sos> ce n est pas moi . <eos>


### 단어 단위 토큰화
- ```Tokenizer```의 인자 중 ```char_level```은 default값으로 False이다.
- 이 인자를 True로 사용한다면 글자 단위의 토큰화를 수행

In [7]:
eng_tokenizer = Tokenizer(filters="", lower=False)            # 토큰화 수행 : 문자 단위 X
eng_tokenizer.fit_on_texts(lines.eng)   # 33000개의 데이터 각 행을 토큰화
input_text = eng_tokenizer.texts_to_sequences(lines.eng)   # 단어를 숫자값 인덱스로 변환
input_text[:3]

[[30, 1], [1132, 1], [1132, 1]]

In [8]:
fra_tokenizer = Tokenizer(filters="", lower=False)
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:3]

[[1, 91, 12, 2], [1, 1068, 12, 2], [1, 1068, 3, 2]]

### 단어장의 사이즈를 저장

In [9]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print("영어 단어장의 크기 : ", eng_vocab_size)
print("프랑스어 단어장의 크기: ", fra_vocab_size)

영어 단어장의 크기 :  4662
프랑스어 단어장의 크기:  7326


### 패딩추가를 위해서 최대 길이를 저장

### 디코더의 데이터 수정

주의!  
- 디코더의 입력에는 ```<eos>``` 토큰이 필요없음
- 디코더의 출력과 비교할 시퀀스는 ```<sos>```가 필요

In [10]:
encoder_input = input_text

# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text]
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [11]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 91, 12], [1, 1068, 12], [1, 1068, 3]]
[[91, 12, 2], [1068, 12, 2], [1068, 3, 2]]


In [12]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [13]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이 : ', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이 : ', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4662
프랑스어 단어장의 크기 : 7326
영어 시퀀스의 최대 길이 :  8
프랑스어 시퀀스의 최대 길이 :  17


### 패딩 추가

패딩을 수행해줘야 한다.  
```max_eng_seq_len```, ```max_fra_seq_len```을 활용해서 해당 길이보다 짧다면 패딩을 채워주는 것

In [14]:
encoder_input = pad_sequences(encoder_input, maxlen=max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [15]:
eng_to_index = eng_tokenizer.word_index
index_to_eng = eng_tokenizer.index_word

fra_to_index = fra_tokenizer.word_index
index_to_fra = fra_tokenizer.index_word

### 데이터셋 나누기
- 데이터를 나누기 전에 먼저 한번 섞어준 후에 Training 3만개, Test 3천개로 나눈다.

In [16]:
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [17]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[ 6045 17112 28339 ...  2455 25641   860]


In [18]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [19]:
encoder_input_train = encoder_input[:30000]
decoder_input_train = decoder_input[:30000]
decoder_target_train = decoder_target[:30000]

encoder_input_test = encoder_input[30000:]
decoder_input_test = decoder_input[30000:]
decoder_target_test = decoder_target[30000:]

In [20]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


## 모델 훈련

### 필요한 라이브러리 import

In [21]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [22]:
# LSTM의 출력 차원
latent_dim = 32

### 인코더 설계  
Masking은 패딩 토큰의 숫자 0의 경우에는 연산을 제외하는 역할을 수행

In [23]:
# 인코더 설계
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, latent_dim)(encoder_inputs) # 임베딩층
enc_masking = Masking(mask_value=0.0)(enc_emb)  # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True)  # 상태값 리턴
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)  # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c]  # 인코더의 은닉 상태외 셀 상태를 저장

### 디코더 설계

- ```optimizer``` : ```rmsprop```
- ```loss``` : ```sparse_categorical_crossentropy```
- ```mterics``` : ```acc```

In [24]:
# 디코더
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(fra_vocab_size, latent_dim)  # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs)  # 패딩 0은 언제나 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequence는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

# 인코더의 은닉 상태를 초기 은닉 상태 (initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점에 결과에 대해 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### 학습

In [25]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 32)     149184      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     234432      input_2[0][0]                    
______________________________________________________________________________________________

In [26]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
235/235 [==============================] - 9s 38ms/step - loss: 3.8890 - acc: 0.6101 - val_loss: 2.1206 - val_acc: 0.6128
Epoch 2/50
235/235 [==============================] - 7s 32ms/step - loss: 1.8707 - acc: 0.6770 - val_loss: 1.7175 - val_acc: 0.7273
Epoch 3/50
235/235 [==============================] - 7s 32ms/step - loss: 1.6282 - acc: 0.7367 - val_loss: 1.5773 - val_acc: 0.7450
Epoch 4/50
235/235 [==============================] - 8s 32ms/step - loss: 1.5250 - acc: 0.7496 - val_loss: 1.5039 - val_acc: 0.7534
Epoch 5/50
235/235 [==============================] - 8s 32ms/step - loss: 1.4530 - acc: 0.7615 - val_loss: 1.4327 - val_acc: 0.7681
Epoch 6/50
235/235 [==============================] - 8s 33ms/step - loss: 1.3869 - acc: 0.7754 - val_loss: 1.3781 - val_acc: 0.7788
Epoch 7/50
235/235 [==============================] - 7s 32ms/step - loss: 1.3346 - acc: 0.7837 - val_loss: 1.3377 - val_acc: 0.7844
Epoch 8/50
235/235 [==============================] - 7s 32ms/step - 

## 모델 테스트

#### 인코더

In [27]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 32)          149184    
_________________________________________________________________
masking (Masking)            (None, None, 32)          0         
_________________________________________________________________
lstm (LSTM)                  [(None, 32), (None, 32),  8320      
Total params: 157,504
Trainable params: 157,504
Non-trainable params: 0
_________________________________________________________________


#### 디코더
- h, c의 차이는 무엇인가, LSTM의 반환되는 것이 뭐지? : ```return_sequence```, ```return_state```
    - 위에서 설정한 LSTM 레이어에서는 sequence와 state를 return해주도록 인자를 주었음
        - ```decoder_outputs``` : 전체 sequence의 output
        - ```state_h2``` : memory.. state..에 대한 정보가 return 되는 것인가?
        - ```state_c2``` : final_carry_state를 리턴하는 것 같은데 이건 뭐지..
    - [TensorFlow 공식 문서 참고 링크](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM#for_example_2)
- 인코더와 디코더의 임베딩층은 서로 다른 임베딩 층을 사용
- 디코더의 훈련과 테스트는 같은 임베딩 층을 사용

In [28]:
# 디코더 설계
# 이전 시점의 상태롤 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# train 때 사용했던 임베딩 층을 재사용..
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단여 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [29]:
# 디코더 정의
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_fra[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

### 결과 확인을 위한 함수

In [31]:
def seq2eng(input_seq):
    temp = ''
    for i in input_seq :
        if(i!=0):
            temp = temp + index_to_eng[i] + ' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2fra(input_seq):
    temp = ''
    for i in input_seq:
        if ((i!=0 and i!=fra_to_index['<sos>']) and i!=fra_to_index['<eos>']):
            temp = temp + index_to_fra[i] + ' '
    return temp

훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플 결과를 출력

In [32]:
for seq_index in [3, 50, 100, 600, 2005]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("원문 : ", seq2eng(encoder_input_train[seq_index]))
    print("번역문 : ", seq2fra(decoder_input_train[seq_index]))
    print("예측문 : ", decoded_sentence[:-5])
    print('\n')

원문 :  i don t like my job . 
번역문 :  je n aime pas mon boulot . 
예측문 :   je n aime pas mon r ponse . 


원문 :  don t talk nonsense . 
번역문 :  ne dis pas de sottises . 
예측문 :   ne me sont pas des argent . 


원문 :  it was rather funny . 
번역문 :  ce fut plut t amusant . 
예측문 :   c tait assez ? 


원문 :  i have no friends . 
번역문 :  je n ai pas d amis . 
예측문 :   je n ai pas de enfants . 


원문 :  life is crazy . 
번역문 :  la vie est dingue . 
예측문 :   la vie est s . 




# 정리
---
1. 텍스트 전처리
    ```python
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)   # 1
    sentence = re.sub(r"[^a-zA-Z!.?]+", r" ", sentence)  # 2
    sentence = re.sub(r"\s+", r" ", sentence)            # 3
    
    sentence = sentence.lower()                          # 4
    ```
    - 1 : 구분자 앞뒤로 공백을 추가
    - 2 : 소문자, 대문자 알파벳, 구분자를 제외한 문자 공백으로 치환
    - 3 : 여러 공백문자들을 하나의 공백으로 치환
    - 4 : 문장을 소문자로 바꿔준다
    
2. 학습 진행
    ```
    Epoch 1/50
235/235 [==============================] - 9s 38ms/step - loss: 3.8890 - acc: 0.6101 - val_loss: 2.1206 - val_acc: 0.6128
Epoch 2/50
235/235 [==============================] - 7s 32ms/step - loss: 1.8707 - acc: 0.6770 - val_loss: 1.7175 - val_acc: 0.7273
    .
    .
    .
    Epoch 49/50
235/235 [==============================] - 8s 33ms/step - loss: 0.7837 - acc: 0.8629 - val_loss: 0.9349 - val_acc: 0.8434
Epoch 50/50
235/235 [==============================] - 8s 33ms/step - loss: 0.7795 - acc: 0.8639 - val_loss: 0.9354 - val_acc: 0.8445
    ```
    validation loss가 안정적으로 떨어지면서 0.9354까지 떨어지면서 학습 중간에 오르기 시작하는 현상(Overfitting)은 관찰되지 않았다.
    
3. 테스트용 디코더의 경우 input을 2개로 받아오면서 이전에 학습할때 사용한 디코더와는 구조가 다르다.
    - 원래 들어오는 input + input의 상태
    - 이전 시점에서 나오는 outputs과 그 상태